In [1]:
import torch

In [2]:
from model import GptLanguageModel
from common import GptConfig

/home/clankur/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
hyperparameters = GptConfig()

In [4]:
model = GptLanguageModel(hyperparameters)
model.load_state_dict(torch.load('model_weights.pth'))
m = model.to(model.device)

In [5]:
tokenizer = m.tokenizer

In [6]:
start_str = """Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. 
It was sad because it couldn't move. Every day, it would say"""
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)

In [7]:
print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))

Once upon a time there was a pumpkin. It was a very special pumpkin, it could speak. 
It was sad because it couldn't move. Every day, it would say goodbye and be safe and safe to others.  It never got lost.
It always remembered that it was about being brave, like a big pumpkin, who lived on it! Every time he was very careful when she played and make sure everyone around the flower. They laughed so high, and it always stayed safe forever forever forever in the future - it was the best day to have fun with the pumpkin! The end!
And it and the pumpkin were the most happy, so lucky to have someone else a safe world around it together and the sun.   It knew that the best day of being kind and helping people could be safe and safe and help everyone. 

 the town knew it was very good to have something new. They were the best way and they all knew the day to be friends and never let the flower come away again and no more knew how much they had to worry. And the end was no longer lonely because

In [10]:
from common import TinyStoriesLoader

# evaluate the model on validation dataset
model.eval()
validation_set = TinyStoriesLoader(hyperparameters, split='validation')
with torch.no_grad():
    val_loss = 0
    i = 0
    while True:
        try:
            inputs = next(validation_set)
            xb, yb = inputs['input_ids'][:, :-1], inputs['input_ids'][:, 1:]
            logits, loss, _ = model(xb.to(m.device), yb.to(m.device))
            val_loss += loss
            i += 1
        except StopIteration:
            break       
    print(val_loss / i) 



Repo card metadata block was not found. Setting CardData to empty.


tensor(1.9497, device='cuda:0')


In [8]:
start_str = """Once upon a time there was a boy named Gin. He was a very big guy"""
encoded_input = tokenizer.encode(start_str)
idx = torch.tensor(encoded_input, dtype=torch.long, device=model.device).unsqueeze(0)

In [9]:

print(tokenizer.decode(m.generate(idx = idx, max_new_tokens=model.block_size-len(encoded_input))[0].tolist()))

Once upon a time there was a boy named Gin. He was a very big guy with a smile and he wanted something very good to eat.
He took his friend and said to him, 'This will be so tasty,' he said with a little hug on. He ran around looking in awe and he felt like it seemed very nice to go. Suddenly his friend Jack heard the voice again! She was so excited!  They were so excited that their friend said 'I am going for an hour!" They laughed, but soon their mom came and asked if she had enough.
The little buddy said yes! He ran to the car, excited to go on the way home he had done something. But he forgot to be very sorry for his mom.
The man looked at the boy and smiled. It felt like the boy had been very kind, and the man said 'I will come and visit your new one!'

Mama hugged his back. The man hugged his son and they both smiled and went back to the same restaurant to enjoy his day with his friends and family, happy. From that the next place was their favorite place to have a new home. The b